In [1]:
import rdkit

from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import MCS
from rdkit.Chem import rdFMCS
from rdkit.Chem import Draw
from rdkit.Chem import *
IPythonConsole.ipython_useSVG=False  

import os
from os import listdir
from os.path import isfile, join

from PIL import Image, ImageDraw, ImageFont
import PIL

import numpy as np
from IPython.display import Image 
import math

from copy import deepcopy

from rdkit.Chem import AllChem, Draw, rdFMCS
from rdkit.Chem.Draw import rdMolDraw2D
from rdkit.Chem.Draw import IPythonConsole

from IPython.display import SVG


C:\Users\josef\AppData\Local\Temp/ipykernel_27300/4055684996.py:5: DeprecationWarning: The rdkit.Chem.MCS module is deprecated; please use rdkit.Chem.rdFMCS instead.
  from rdkit.Chem import MCS


In [2]:
import networkx as nx
import logging
logger = logging.getLogger(__name__)

# General function for finding the common core of two molecules, creating dictionaries, converting rdkit-mol to networkx-graph etc. 

#### create additional dictionaries/properties for a mol object (the goal is identity between the mol-objects of the notebook and Transformato):

In [3]:
def generate_apply_dicts(
    mol
) :
    """
    Generate mapping dictionaries for a molecule in a psf.
    Parameters
    ----------
    psf: pm.charmm.CharmmPsfFile
    Returns
    ----------
    dict's
    """

    atom_idx_to_atom_name = dict()
    atom_name_to_atom_idx = dict()
    atom_name_to_atom_type = dict()
    atom_idx_to_atom_partial_charge = dict()

    for atom in mol.GetAtoms():
        atom_name = atom.GetSymbol()
      #  atom_name = atom.GetAtomicNum()
        atom_index = atom.GetIdx()
        atom_type = atom.GetSymbol()
        atom_charge = atom.GetFormalCharge()

        atom_idx_to_atom_name[atom_index] = atom_name
        atom_name_to_atom_idx[atom_name] = atom_index
        atom_name_to_atom_type[atom_name] = atom_type
        atom_idx_to_atom_partial_charge[atom_index] = atom_charge

    for atom in mol.GetAtoms():
 
        atom.SetProp("atom_name", atom_idx_to_atom_name[atom.GetIdx()])
        atom.SetProp(
            "atom_type",
            atom_name_to_atom_type[atom_idx_to_atom_name[atom.GetIdx()]],
        )
        atom.SetProp("atom_index", str(atom.GetIdx()))
        atom.SetProp(
            "atom_charge", str(atom_idx_to_atom_partial_charge[atom.GetIdx()])
        )
    return mol




### diverse functions for converting rdkit-mol object to networkx-graph object

In [4]:
def mol_to_nx(mol):
    G = nx.Graph()

    for atom in mol.GetAtoms():
        G.add_node(atom.GetIdx(),
                   atomic_num=atom.GetAtomicNum(),
                   is_aromatic=atom.GetIsAromatic(),
                   atom_symbol=atom.GetSymbol())
        
    for bond in mol.GetBonds():
        G.add_edge(bond.GetBeginAtomIdx(),
                   bond.GetEndAtomIdx(),
                   bond_type=bond.GetBondType())
        
    return G

In [5]:
def _mol_to_nx_full(mol):
    G = nx.Graph()


    #to get same representation in networkx as in Transformato (copied from mutate.py)
    atoms = mol.GetNumAtoms()
    for idx in range(atoms):
        mol.GetAtomWithIdx(idx).SetProp(
            "molAtomMapNumber2", str(mol.GetAtomWithIdx(idx).GetIdx())
        )

    for atom in mol.GetAtoms():
        G.add_node(
            atom.GetIdx(),
            atomic_num=atom.GetAtomicNum(),
            formal_charge=atom.GetFormalCharge(),
            chiral_tag=atom.GetChiralTag(),
            hybridization=atom.GetHybridization(),
            num_explicit_hs=atom.GetNumExplicitHs(),
            is_aromatic=atom.GetIsAromatic(),
          
            atomic_id = atom.GetIdx(),
            atom_symbol=atom.GetSymbol(),

            atom_map = atom.GetProp("molAtomMapNumber2"),
            
            #additionally

            atom_index = atom.GetProp("atom_index"),

            atom_type = atom.GetProp("atom_type"),

            atom_index_type = str(atom.GetProp("atom_type")) + ":" + str(atom.GetProp("atom_index"))


          #  str(i.GetProp("atom_index")) + ":" + i.GetProp("atom_type")

        )

    for bond in mol.GetBonds():
        G.add_edge(
            bond.GetBeginAtomIdx(),
            bond.GetEndAtomIdx(),
            bond_type=bond.GetBondType(),
        )
    return G

In [6]:
#also weights are added (necessary for new mutation algorithms)
def _mol_to_nx_full_weight(mol):
    G = nx.Graph()


    #to get same representation in networkx as in transformato(copied from mutate.py)
    atoms = mol.GetNumAtoms()
    for idx in range(atoms):
        mol.GetAtomWithIdx(idx).SetProp(
            "molAtomMapNumber2", str(mol.GetAtomWithIdx(idx).GetIdx())
        )

    for atom in mol.GetAtoms():
        G.add_node(
            atom.GetIdx(),
            atomic_num=atom.GetAtomicNum(),
            formal_charge=atom.GetFormalCharge(),
            chiral_tag=atom.GetChiralTag(),
            hybridization=atom.GetHybridization(),
            num_explicit_hs=atom.GetNumExplicitHs(),
            is_aromatic=atom.GetIsAromatic(),
           
            atomic_id = atom.GetIdx(),
            atom_symbol=atom.GetSymbol(),

            atom_map = atom.GetProp("molAtomMapNumber2"),
            
            #additionally

            atom_index = atom.GetProp("atom_index"),

            atom_type = atom.GetProp("atom_type"),

            atom_index_type = str(atom.GetProp("atom_type")) + ":" + str(atom.GetProp("atom_index"))


          #  str(i.GetProp("atom_index")) + ":" + i.GetProp("atom_type")

        )

    for bond in mol.GetBonds():
        G.add_edge(
            bond.GetBeginAtomIdx(),
            bond.GetEndAtomIdx(),
            bond_type=bond.GetBondType(),
            
            weight = 5
        )
    return G

## get the common core of two molecules (rdkit-mols)

In [7]:
def get_common_core(mol1, mol2):
    mols = [mol1, mol2]
    
    res=rdFMCS.FindMCS(mols, ringMatchesRingOnly = True, completeRingsOnly = True, ringCompare=rdkit.Chem.rdFMCS.RingCompare.StrictRingFusion )    
    substructure = Chem.MolFromSmarts(res.smartsString)
    #res=rdFMCS.FindMCS(mols, ringMatchesRingOnly = True, completeRingsOnly = True )    

    
    ccore = MolFragmentToCXSmiles(mol1, mol1.GetSubstructMatch(substructure), kekuleSmiles = True, isomericSmiles=False)
    ccoremol = Chem.MolFromSmiles(ccore)
    
    hit_ats1 = mol1.GetSubstructMatch(substructure)
    hit_bonds1 = []
    for bond in substructure.GetBonds():
        aid1 = hit_ats1[bond.GetBeginAtomIdx()]
        aid2 = hit_ats1[bond.GetEndAtomIdx()]
        hit_bonds1.append(mol1.GetBondBetweenAtoms(aid1,aid2).GetIdx())
    

    hit_ats2 = mol2.GetSubstructMatch(substructure)
    hit_bonds2 = []
    for bond in substructure.GetBonds():
        aid1 = hit_ats2[bond.GetBeginAtomIdx()]
        aid2 = hit_ats2[bond.GetEndAtomIdx()]
        hit_bonds2.append(mol2.GetBondBetweenAtoms(aid1,aid2).GetIdx())
        
    
    mol1coreindex = [hit_ats1]
    mol2coreindex= [hit_ats2]
    
    return mol1coreindex, mol2coreindex, hit_ats1, hit_ats2
    

## functions prior to the mutation route step: find connected dummy regions and terminal atoms, etc.
### mostly from Transformato, however slightly modified (especially no reference to proposed_mutation_route and systemstructure objects)

In [8]:
def _find_connected_dummy_regions_mol(mol, ccore, G):

 
    sub=ccore
    G_dummy = G.copy()
    
    
    from itertools import chain
    sub = [int(x) for x in chain.from_iterable(ccore)]
  
    # find all dummy atoms
    list_of_dummy_atoms_idx = [
        atom.GetIdx() for atom in mol.GetAtoms() if atom.GetIdx() not in sub
    ]
    nr_of_dummy_atoms = len(list_of_dummy_atoms_idx) + 1
    list_of_real_atoms_idx = [
        atom.GetIdx() for atom in mol.GetAtoms() if atom.GetIdx() in sub
    ]
    
    print("list_of_dummy_atoms_idx")
    print(list_of_dummy_atoms_idx)

    print("nr_of_dummy_atoms")
    print(nr_of_dummy_atoms)
    
    print("list_of_real_atoms_idx")
    print(list_of_real_atoms_idx)

    
    # remove real atoms from graph to obtain multiple connected compounds
    for real_atom_idx in list_of_real_atoms_idx:
        G_dummy.remove_node(real_atom_idx)

    # find these connected compounds
    from networkx.algorithms.components import connected_components

    unique_subgraphs = [
        c for c in sorted(nx.connected_components(G_dummy), key=len, reverse=True)
    ]
    
    return unique_subgraphs, G_dummy

In [9]:
def _find_terminal_atom(cc_idx, mol):
    """
    Find atoms that connect the molecule to the common core.

    Args:
        cc_idx (list): common core index atoms
        mol ([type]): rdkit mol object
    """
    terminal_dummy_atoms = []
    terminal_real_atoms = []
    
    from itertools import chain
    cc_idx = [int(x) for x in chain.from_iterable(cc_idx)]

    for atom in mol.GetAtoms():
        idx = atom.GetIdx()
      
        
        if idx not in cc_idx:
            neighbors = [x.GetIdx() for x in atom.GetNeighbors()]
            if any([n in cc_idx for n in neighbors]):
                terminal_dummy_atoms.append(idx)
        if idx in cc_idx:
            neighbors = [x.GetIdx() for x in atom.GetNeighbors()]
            if any([n not in cc_idx for n in neighbors]):
                terminal_real_atoms.append(idx)

    logger.info(f"Terminal dummy atoms: {str(list(set(terminal_dummy_atoms)))}")
    logger.info(f"Terminal real atoms: {str(list(set(terminal_real_atoms)))}")

    return (list(set(terminal_dummy_atoms)), list(set(terminal_real_atoms)))

In [10]:
def _match_terminal_real_and_dummy_atoms(mol, real_atoms_cc, dummy_atoms_cc):
#      mol, real_atoms_cc: list, dummy_atoms_cc: list
#   ) -> dict:
    """
    Matches the terminal real and dummy atoms and returns a dict with real atom idx as key and a set of dummy atoms that connect
    to this real atom as a set

    Parameters
    ----------
    mol : [Chem.Mol]
        The mol object with the real and dummy atoms
    real_atoms_cc : list
        list of real atom idx
    dummy_atoms_cc : list
        list of dummy atom idx

    Returns
    -------
    [type]
        [description]
    """

    from collections import defaultdict

    real_atom_match_dummy_atom = defaultdict(set)
    for real_atom_idx in real_atoms_cc:
        real_atom = mol.GetAtomWithIdx(real_atom_idx)
        real_neighbors = [x.GetIdx() for x in real_atom.GetNeighbors()]
        for dummy_atoms_idx in dummy_atoms_cc:
            if dummy_atoms_idx in real_neighbors:
                real_atom_match_dummy_atom[real_atom_idx].add(dummy_atoms_idx)
                

    return real_atom_match_dummy_atom

###  fully new function: again check for connected components after matching real and dummy atoms (only necessary for pathological cases where two atoms of the same component are connected to the common core)


In [11]:
def reduce_terminal(match_terminal_atoms, subg, G_dummy):
    #new: again check for connected components (only necessary for pathological cases where two atoms of the same component are connected to the common core)
    from networkx.algorithms.components import connected_components

    coreconnectingnodes = []
    
    for nodes in match_terminal_atoms.values():
        aequinodes = []
     
        for node in nodes:
            coreconnectingnodes.append(node)
          
        for i in range(len(coreconnectingnodes)):
        
            for j in range(i, len(coreconnectingnodes)):
            
                if (i != j ):
                   
                    if (nx.has_path(G_dummy,coreconnectingnodes[i],coreconnectingnodes[j])):
                       
                        nodes.remove(coreconnectingnodes[i])
                                  
        coreconnectingnodes = []
    
    return match_terminal_atoms

# Mutation Route Algorithms (i.e. the most important part)

## 3 main approaches (_calculate_order_of_LJ_mutations, _calculate_order_of_LJ_mutations_new and _calculate_order_of_LJ_mutations_iter):
### _calculate_order_of_LJ_mutations: naive dfs (as currently in transformato)
### _calculate_order_of_LJ_mutations_new: bfs/djikstra-algorithm applied once for route
### _calculate_order_of_LJ_mutations_new_iter: bfs/djikstra-algorithm applied iteratively, i.e. after each removal of an atom 

In [12]:
def _calculate_order_of_LJ_mutations(
    connected_dummy_regions: list, match_terminal_atoms: dict, G: nx.Graph
) -> list:

    ordered_LJ_mutations = []
    for real_atom in match_terminal_atoms:
        for dummy_atom in match_terminal_atoms[real_atom]:
            for connected_dummy_region in connected_dummy_regions:
                # stop at connected dummy region with specific dummy_atom in it
                if dummy_atom not in connected_dummy_region:
                    continue

                G_dummy = G.copy()
                # delete all nodes not in dummy region
                remove_nodes = [
                    node for node in G.nodes() if node not in connected_dummy_region
                ]
                for remove_node in remove_nodes:
                    G_dummy.remove_node(remove_node)

                # root is the dummy atom that connects the real region with the dummy region
                root = dummy_atom

                edges = list(nx.dfs_edges(G_dummy, source=root))
                nodes = [root] + [v for u, v in edges]
                nodes.reverse()  # NOTE: reverse the mutation
                ordered_LJ_mutations.append(nodes)

    return ordered_LJ_mutations

In [13]:
def _calculate_order_of_LJ_mutations_new(
    connected_dummy_regions: list, match_terminal_atoms: dict, G: nx.Graph, cyclecheck=True, ordercycles=True
) -> list:

    ordered_LJ_mutations = []
    
    final_orders = []
    
    
    for real_atom in match_terminal_atoms:
        for dummy_atom in match_terminal_atoms[real_atom]:
            for connected_dummy_region in connected_dummy_regions:
                # stop at connected dummy region with specific dummy_atom in it
                if dummy_atom not in connected_dummy_region:
                    continue

                G_dummy = G.copy()
                # delete all nodes not in dummy region
                remove_nodes = [
                    node for node in G.nodes() if node not in connected_dummy_region
                ]
                for remove_node in remove_nodes:
                    G_dummy.remove_node(remove_node)

                # root is the dummy atom that connects the real region with the dummy region
                root = dummy_atom
                
                #process cycles
                if (cyclecheck == True and ordercycles==False):
                    G_dummy = cycle_checks_nx(G_dummy)
                
                #process cycles and correct order (according to 'preferential removal')
                if (cyclecheck == True and ordercycles==True):
                    cycledict, degreedict = cycle_checks(G_dummy)
                             
                        
                #dijkstra          
                ssource = nx.single_source_dijkstra(G_dummy, source=root, weight='weight')
     
                sortedssource = {k: v for k, v in sorted(ssource[0].items(), key=lambda item: item[1])}   
    
                max_node = max(ssource[0], key=ssource[0].get)
                      
                sortedssource_edges = sortedssource.keys()

                sortedssource_edges_list = list(sortedssource_edges)
                 
                nodes = sortedssource_edges_list
            
                nodes.reverse()  
                
                #sort nodes according to degree, cycle participation and removal order
                if (cyclecheck == True and ordercycles==True):
                    nodes = change_route_cycles(nodes, cycledict, degreedict, sortedssource, G)
             
                
                print("Final mutation route:")
                print(nodes)
                ordered_LJ_mutations.append(nodes)

    return ordered_LJ_mutations

In [14]:
def _calculate_order_of_LJ_mutations_new_iter(
    connected_dummy_regions: list, match_terminal_atoms: dict, G: nx.Graph, cyclecheck=True, ordercheck=True
) -> list:

    ordered_LJ_mutations = []
    
    final_orders = []
    
    
    
    for real_atom in match_terminal_atoms:
        for dummy_atom in match_terminal_atoms[real_atom]:
            for connected_dummy_region in connected_dummy_regions:
                # stop at connected dummy region with specific dummy_atom in it
                if dummy_atom not in connected_dummy_region:
                    continue

                G_dummy = G.copy()
                # delete all nodes not in dummy region
                remove_nodes = [
                    node for node in G.nodes() if node not in connected_dummy_region
                ]
                for remove_node in remove_nodes:
                    G_dummy.remove_node(remove_node)

                # root is the dummy atom that connects the real region with the dummy region
                root = dummy_atom

           
                
                final_order = []
                
                removeG = nx.Graph()
                removearray = []
                while ( len(G_dummy.nodes())  > 0):
                    #update weights according to already removed nodes
                    if (ordercheck == True):
                        G_dummy = order_checks_nx(G_dummy, removearray, G)
                    
                    G_origweights = G_dummy.copy()
                    
                    #update weights according to cycle participation
                    if (cyclecheck == True):
                        G_dummy = cycle_checks_nx(G_dummy)
                   

                    
                    #dijkstra    
                    ssource = nx.single_source_dijkstra(G_dummy, source=root, weight='weight')

                    sortedssource = {k: v for k, v in sorted(ssource[0].items(), key=lambda item: item[1])}
                    max_node = max(ssource[0], key=ssource[0].get)
            
                    final_order.extend([max_node])
                    
                    
                    #restore original weights
                    G_dummy = G_origweights
                    
                    #remove G_dummy
                    G_dummy.remove_node(max_node)
                    
                    #add to removeG
                    removeG.add_node(max_node)
                    removearray.append(max_node)
                    
                    sortedssource_edges = final_order
                    
              
                #sortedssource_edges_list already contains the nodes in right (reversed) order (including root)
                nodes = sortedssource_edges
                
                print("Final mutation route:")
                print(nodes)
                
                #no reverse (already correct order, starting with the node with greatest distance from root)
                ordered_LJ_mutations.append(nodes)
                
               

    return ordered_LJ_mutations

### additional functions for the new mutation route algorithms (using weight updates, cycle and degree functions of networkx): process cycles (atoms participating in many cycles are removed later) and 'preferential removal' (atoms which neighbours already have been removed are removed earlier)

In [15]:
#cycle processing, currently used in _calculate_order_of_LJ_mutations_new_iter 
def cycle_checks_nx(G):
    
    #search cycles using networkx
    cycles = nx.cycle_basis(G)
    
    import collections
    from collections import Counter
    
    cdict = Counter(x for xs in cycles for x in set(xs))
  
    
    #modify weighted graph: nodes participating in many cycles get lower weight
    for i in cdict:
        edg = G.edges(i)
        for el in edg:
            G[el[0]][el[1]]['weight'] = G[el[0]][el[1]]['weight'] - cdict[i]*0.5
    
    return G

In [16]:
#preferential removal, currently used in _calculate_order_of_LJ_mutations_new_iter 
def order_checks_nx(G, removearray, G_total):
    if (len(removearray) > 0):
        lastremoved = removearray[len(removearray) - 1]
        
        edg = G_total.edges(lastremoved)
        
        edg_dummy = G.edges()
        
        for ed in edg:
       
            if (ed[0]!=lastremoved):
                connectednode = ed[0]
            else:
                connectednode = ed[1]
            
            #if node is connected to last removed node, its weight get a small increase
            if (G.has_node(connectednode)):
             
                connectededges = G.edges(connectednode)
                for el in connectededges:
                   
                    G[el[0]][el[1]]['weight'] = G[el[0]][el[1]]['weight'] + 0.1
                                    
            
    return G

In [17]:
#cycle processing dictionary and degree dictionary for preferential removal, currently used in _calculate_order_of_LJ_mutations_new (via change_route_cycles)
def cycle_checks(G):
    
    
    #search cycles using networkx
    cycles = nx.cycle_basis(G)
    
    #alternatively, using rdkit
    #ri = mol.GetRingInfo()
    #cyclesrdkit = ri.AtomRings()
    
    import collections
    from collections import Counter
    
    cdict = Counter(x for xs in cycles for x in set(xs))
    #cdictrdkit = Counter(x for xs in cyclesrdkit for x in set(xs))
    
    #add atoms with no cycle participation
    for key in G.nodes:
        if key not in cdict:
            cdict[key] = 0
    
    degreedict = G.degree()
    degreedict = {node:val for (node, val) in degreedict}      
    
    return cdict, degreedict

In [18]:
#currently used in _calculate_order_of_LJ_mutations_new
#preliminary mutation is list is sorted using cycle and degree dictionary
def change_route_cycles(route, cycledict, degreedict, weightdict, G):

    
    for i in range(len(route)-1):
        routedict = route[i]
        routeweight = weightdict.get(route[i])
        
        routecycleval = cycledict.get(route[i])
        routedegreeval = degreedict.get(route[i])
       
        for j in range(i, len(route)):
       
            if (routeweight == weightdict[route[j]]):
                      
                #if nodes have same weight (i.e. distance from root), the node participating in more cycles is removed later
                
                if (routecycleval > cycledict[route[j]] or (routecycleval == cycledict[route[j]] and routedegreeval > degreedict[route[j]])):
                    idx1 = route.index(route[i])
                    idx2 = route.index(route[j])                  
                    route[idx1], route[idx2] = route[idx2], route[idx1]                 
                    continue
                    
                #if nodes have same weight (i.e. distance from root) and same cycle participation number, the node which has more neighbours already removed is removed earlier

                
                if (routecycleval == cycledict[route[j]]):
                    
                    edgesi = G.edges(routedict)
                    edgesj = G.edges(route[j])

                    iedgecounter = 0
                    for edge in edgesi:
                        if (edge[1] in route[0:i] or edge[0] in route[0:i] ):

                            iedgecounter = iedgecounter + 1

                    jedgecounter = 0
                    for edge in edgesj:
                        if (edge[1] in route[0:i] or edge[0] in route[0:i]):

                            jedgecounter = jedgecounter + 1


                    if (iedgecounter < jedgecounter):
                        idx1 = route.index(route[i])
                        idx2 = route.index(route[j])
                        route[idx1], route[idx2] = route[idx2], route[idx1]

    
    return route

# Visualization functions

### used for color gradient

In [19]:
from colorsys import hls_to_rgb

def rainbow_color_gradient(n=10, end = 1/3):
  #  end=n/40
  #  end = 1/3
    if (n > 1):
        return [ hls_to_rgb(end * i/(n-1), 0.5, 1) for i in range(n) ]
    else:
        return [ hls_to_rgb(end * i/(n), 0.5, 1) for i in range(n) ]

### function for drawing the molecule with color gradient for mutation order

In [20]:
def _show_common_core_gradient( mol, highlight, mutationl, percomponent = False, numbers = False):
    
    """
    Draw the molecule with color gradient for mutation order
    
    highlight: list indicating the ccore, fct highlights the common core in black
    mutationl: list indicating the mutation order, fct draws color gradient
    
    percompoentn: boolean; one color gradient for each component (False) or one for all dummy atoms
    - currently, the components are processed successively, so perhaps True is better suited to illustrate the mutation order
    
    numbers: boolean; show also numbers showing the mutation order
    """
  
    # https://rdkit.blogspot.com/2015/02/new-drawing-code.html

    mol = deepcopy(mol)
    AllChem.Compute2DCoords(mol)

    drawer = rdMolDraw2D.MolDraw2DSVG(800, 800)


    #new
    drawer.drawOptions().setHighlightColour((0.8,0.8,0.8))

    drawer.SetFontSize(0.3)

    opts = drawer.drawOptions()

    for i in mol.GetAtoms():
        opts.atomLabels[i.GetIdx()] = (
            str(i.GetProp("atom_index")) + ":" + i.GetProp("atom_type")
        )

    colours = [(0.8,0.0,0.8),(0,0.15,0),(0,0.8,0.8),(0,0,0.8)]
    atom_cols = {}
    for i, at in enumerate(highlight): 
        atom_cols[at] = colours[1]
    
    if (percomponent == False):
        mutationlflat =  [item for sublist in mutationl for item in sublist]
       
        #colour gradient
        rcolours = rainbow_color_gradient(n=len(mutationlflat))

        for i, at in enumerate(mutationlflat):
            atom_cols[at] = rcolours[i]
        
        if (numbers == True):
            atomlist = mol.GetAtoms()
            for i, at in enumerate(mutationlflat):           
                mnr = i+1
                lbl = '%.0f'%mnr
                atomlist[at].SetProp('atomNote',lbl)


       
        mutationlflat = mutationlflat + list(highlight)
    
        drawer.DrawMolecule(mol, highlightAtoms=mutationlflat, highlightAtomColors=atom_cols)
        Draw.DrawingOptions.includeAtomNumbers = False
        drawer.FinishDrawing()
        svg = drawer.GetDrawingText().replace("svg:", "")
        
        img = Draw.MolToImage(mol,  highlightAtoms=mutationlflat, highlightAtomColors=atom_cols,  useSVG=False)

    
    else:
        mutationlflat =  [item for sublist in mutationl for item in sublist]

        
        componentcount = 0
        for mutcomp in mutationl:
               
                rcolours = rainbow_color_gradient(n=len(mutcomp))
                for i, at in enumerate(mutcomp):
                   
                    atom_cols[at] = rcolours[i]
                    
                componentcount = componentcount+1
                
                
        if (numbers == True):
            atomlist = mol.GetAtoms()
            for i, at in enumerate(mutationlflat):           
                mnr = i+1
                lbl = '%.0f'%mnr
                atomlist[at].SetProp('atomNote',lbl)
                
        mutationlflat = mutationlflat + list(highlight)

       
        drawer.DrawMolecule(mol, highlightAtoms=mutationlflat, highlightAtomColors=atom_cols)
        Draw.DrawingOptions.includeAtomNumbers = False
        drawer.FinishDrawing()
        svg = drawer.GetDrawingText().replace("svg:", "")
        
        img = Draw.MolToImage(mol,  highlightAtoms=mutationlflat, highlightAtomColors=atom_cols, useSVG=False)


    return svg

## almost identical to function above, used for creating visualizations to write in files 

In [21]:
def _show_common_core_gradient_write( mol, highlight, mutationl, percomponent = False, numbers = False, legendtext = ""):
   

    """
    Draw the molecule with color gradient for mutation order
    
    highlight: list indicating the ccore, fct highlights the common core in black
    mutationl: list indicating the mutation order, fct draws color gradient
    
    percompoentn: boolean; one color gradient for each component (False) or one for all dummy atoms
    - currently, the components are processed successively, so perhaps True is better suited to illustrate the mutation order
    
    numbers: boolean; show also numbers showing the mutation order
    legendtext: string; legend to be placed below molecule
    """
    
    # https://rdkit.blogspot.com/2015/02/new-drawing-code.html

    mol = deepcopy(mol)
    AllChem.Compute2DCoords(mol)

    drawer = rdMolDraw2D.MolDraw2DSVG(600, 600)


    #new
    drawer.drawOptions().setHighlightColour((0.8,0.8,0.8))

    drawer.SetFontSize(0.3)

    opts = drawer.drawOptions()

    for i in mol.GetAtoms():
        opts.atomLabels[i.GetIdx()] = (
            str(i.GetProp("atom_index")) + ":" + i.GetProp("atom_type")
        )


    colours = [(0.8,0.0,0.8),(0,0.15,0),(0,0.8,0.8),(0,0,0.8)]
    atom_cols = {}
    for i, at in enumerate(highlight): 
        atom_cols[at] = colours[1]
    
    if (percomponent == False):
        mutationlflat =  [item for sublist in mutationl for item in sublist]
    
        #rainbow gradient
        rcolours = rainbow_color_gradient(n=len(mutationlflat))
    
        for i, at in enumerate(mutationlflat):
            atom_cols[at] = rcolours[i]

        if (numbers == True):
            atomlist = mol.GetAtoms()
            for i, at in enumerate(mutationlflat):           
                mnr = i+1
                lbl = '%.0f'%mnr
                atomlist[at].SetProp('atomNote',lbl)
    
    
        mutationlflat = mutationlflat + list(highlight)
    
        drawer.DrawMolecule(mol, highlightAtoms=mutationlflat, highlightAtomColors=atom_cols, legend=legendtext)
        Draw.DrawingOptions.includeAtomNumbers = False
        drawer.FinishDrawing()
        svg = drawer.GetDrawingText().replace("svg:", "")
        
        img = Draw.MolToImage(mol,  highlightAtoms=mutationlflat, highlightAtomColors=atom_cols,  useSVG=False)

    
    else:
        mutationlflat =  [item for sublist in mutationl for item in sublist]

        componentcount = 0
        for mutcomp in mutationl:
               
                rcolours = rainbow_color_gradient(n=len(mutcomp))
                for i, at in enumerate(mutcomp):
                   
                    atom_cols[at] = rcolours[i]
                    
                componentcount = componentcount+1
                
        if (numbers == True):
            atomlist = mol.GetAtoms()
            for i, at in enumerate(mutationlflat):           
                mnr = i+1
                lbl = '%.0f'%mnr
                atomlist[at].SetProp('atomNote',lbl)
                
        mutationlflat = mutationlflat + list(highlight)

       
        drawer.DrawMolecule(mol, highlightAtoms=mutationlflat, highlightAtomColors=atom_cols, legend=legendtext)
        Draw.DrawingOptions.includeAtomNumbers = False
        drawer.FinishDrawing()
        svg = drawer.GetDrawingText().replace("svg:", "")
        
        img = Draw.MolToImage(mol,  highlightAtoms=mutationlflat, highlightAtomColors=atom_cols, useSVG=False)
        
    
   
    return svg, img
    


In [22]:
def nxgraph(G):
    mol_attr = nx.get_node_attributes(G, 'atom_index_type')
    #nx.draw(G, with_labels = True)
    nx.draw(G, labels=mol_attr)

### converting files to appropriate format for writing in file

In [23]:
def convertgraphics(img1, img2):
    from cairosvg import svg2png
    from io import BytesIO
    imgn1 = svg2png(img1)
    imgn2 = svg2png(img2)

    import PIL.Image
    imo1 = PIL.Image.open(BytesIO(imgn1))
    imo2 = PIL.Image.open(BytesIO(imgn2))
    
    return imo1, imo2

# Start here!

# Read sdf-files from pdbbind-folder

In [24]:
filecounter = 0
sdfarray=[]
files = os.listdir('pdbbindsdf20/')

In [25]:
#choose subset
#files = files[0:5]

In [26]:
#sdf to smiles
for file in files:
    sdf = Chem.SDMolSupplier('pdbbindsdf20/' + file )
    for i in sdf:
        smisdf = Chem.MolToSmiles(i)
    sdfarray.append(smisdf)

In [27]:
print(sdfarray)

['CCCCCN(CCCCC)C(=O)C(CCC(=O)O)NC(=O)C(Cc1ccccc1)NC(C)=O', 'CCCCC(NC(=O)CCC(=O)O)P(=O)(O)Oc1ccccc1', 'OCC1OC(OC2(CO)OC(CO)C(O)C2O)C(O)C(O)C1O', 'CCCCCN(CCCCC)C(=O)C(CCC(=O)O)NC(=O)C(Cc1ccc(OP(=O)(O)O)cc1)NC(C)=O', 'CC(=O)NC(Cc1ccc(OP(=O)(O)O)cc1)C(=O)NC(CCC(=O)O)C(=O)N(C)CCCC1CCCC1', 'CCCCC1CCCN(C(=O)C(CCC(=O)O)NC(=O)C(Cc2ccc(OP(=O)(O)O)cc2)NC(C)=O)C1', 'CC(C)CC(NC(=O)C(O)Cc1ccc(O)cc1)C(=O)N1C(C(=O)NC(CO)CCCNC(=N)N)CC2CCC(O)CC21', 'CCC(C)C(NC(=O)C(CCC(=O)O)NC(=O)C(CCC(=O)O)NC(=O)C(Cc1ccccc1)NC(=O)C(CC(=O)O)NC(=O)C[NH3+])C(=O)N1CC=CC1C(=O)NCC(=O)NC(CCC(=O)O)C(=O)NC(Cc1ccc(O)cc1)C(=O)NC(C)C(=O)O', 'CC(=O)NC1C(NC(=N)N)C=C(C(=O)O)OC1C(O)C(O)CO', 'COC1=C2CC(C)CC(OC)C(O)C(C)/C=C(\\C)C(OC(N)=O)C(OC)/C=C\\C=C(/C)C(=O)NC(=CC1=O)C2=O', 'CC(=O)Nc1ccc(N2C(=O)C3C4CCC(NC(=O)OCC(=O)O)(CC4)C3C2=O)cc1', 'O=c1[nH]cnc2c1ncn2C1OC(CO)C(O)C1O', 'CCCN(CCc1ccccc1)C(=O)C1OC(C(=O)O)=CC([NH3+])C1NC(C)=O', 'Nc1nc2c(ncn2C2OC(COP(=O)(O)OP(N)(=O)O)C(O)C2O)c(=O)[nH]1', 'CN(C)c1cccc2c(S(=O)(=O)NC(CCCNC(=N)N)C(=O)N3CCC

In [28]:
molarray = []
for sdf in sdfarray:    
    mol = Chem.MolFromSmiles(sdf)
    molarray.append(mol)

In [29]:
molarray2 = molarray.copy()

In [30]:
molnames =[]
for mol1 in files:
    for mol2 in files:
        molnames.append(mol1)
        molnames.append(mol2)

In [31]:
print(molarray)

[<rdkit.Chem.rdchem.Mol object at 0x000002671AB44340>, <rdkit.Chem.rdchem.Mol object at 0x000002671AB44940>, <rdkit.Chem.rdchem.Mol object at 0x000002671AB44220>, <rdkit.Chem.rdchem.Mol object at 0x000002671A6789A0>, <rdkit.Chem.rdchem.Mol object at 0x000002671A678C40>, <rdkit.Chem.rdchem.Mol object at 0x000002671A678A60>, <rdkit.Chem.rdchem.Mol object at 0x000002671AB6E4C0>, <rdkit.Chem.rdchem.Mol object at 0x000002671AB6EA00>, <rdkit.Chem.rdchem.Mol object at 0x000002671AB6E820>, <rdkit.Chem.rdchem.Mol object at 0x000002671AB6E400>, <rdkit.Chem.rdchem.Mol object at 0x000002671AB63D60>, <rdkit.Chem.rdchem.Mol object at 0x000002671AB636A0>, <rdkit.Chem.rdchem.Mol object at 0x000002671AB63DC0>, <rdkit.Chem.rdchem.Mol object at 0x000002671AB63CA0>, <rdkit.Chem.rdchem.Mol object at 0x000002671AB63A00>, <rdkit.Chem.rdchem.Mol object at 0x000002671AB57C40>, <rdkit.Chem.rdchem.Mol object at 0x000002671AB57940>, <rdkit.Chem.rdchem.Mol object at 0x000002671AB579A0>, <rdkit.Chem.rdchem.Mol obje

In [32]:
imagearray = []
imagearraynew = []
imagearraynewiter = []
molnamescounter = 0
for mol1 in molarray:
    for mol2 in molarray:
        
        mol1 = generate_apply_dicts(mol1)
        mol2 = generate_apply_dicts(mol2)

        mol1coreindex, mol2coreindex, hit_ats1, hit_ats2 = get_common_core(mol1, mol2)

        graphmol1 = _mol_to_nx_full_weight(mol1)
        graphmol2 = _mol_to_nx_full_weight(mol2)

        subg1, G_dummy1 = _find_connected_dummy_regions_mol(mol1, mol1coreindex, graphmol1)
        subg2, G_dummy2 = _find_connected_dummy_regions_mol(mol2, mol2coreindex, graphmol2)

        terminaldummy1, terminalreal1 = _find_terminal_atom(mol1coreindex,  mol1)
        terminaldummy2, terminalreal2 = _find_terminal_atom(mol2coreindex,  mol2)

        matchterminal1 = _match_terminal_real_and_dummy_atoms(mol1, terminalreal1, terminaldummy1)
        matchterminal2 = _match_terminal_real_and_dummy_atoms(mol2, terminalreal2, terminaldummy2)

        #not necessary
        #matchterminal1 = reduce_terminal(matchterminal1, subg1, G_dummy1)
        #matchterminal2 = reduce_terminal(matchterminal2, subg2, G_dummy2)

        #old function from transformato (simply dfs)
        order1 = _calculate_order_of_LJ_mutations(
        subg1, matchterminal1, graphmol1
    )
        order2 = _calculate_order_of_LJ_mutations(
        subg2, matchterminal2, graphmol2
    )
        
        #new mutation order function
        order1new = _calculate_order_of_LJ_mutations_new(
        subg1, matchterminal1, graphmol1
    )
        order2new = _calculate_order_of_LJ_mutations_new(
        subg2, matchterminal2, graphmol2
    )
        
        #new mutation order function - code with iteration
        order1newiter = _calculate_order_of_LJ_mutations_new_iter(
        subg1, matchterminal1, graphmol1
    )
        order2newiter = _calculate_order_of_LJ_mutations_new_iter(
        subg2, matchterminal2, graphmol2
    )

        #visualization
        legendleft = "left: dfs; middle: dijkstra/bfs; right: dijkstra/bfs iterated; ccore in dark"
        
        legend1 = "route for: " + molnames[molnamescounter] 
        legend2 = "route for: " + molnames[molnamescounter+1] 

        legendright = "ccore of" + molnames[molnamescounter] + "and" + molnames[molnamescounter+1]
       
        
        svmol1, img1 = _show_common_core_gradient_write( mol1, hit_ats1, order1, True, True, legendleft)
        svmol2, img2 = _show_common_core_gradient_write( mol2, hit_ats2, order2, True, True, legendleft)
        
        svmol1new, img1new = _show_common_core_gradient_write( mol1, hit_ats1, order1new, True, True, legend1)
        svmol2new, img2new = _show_common_core_gradient_write( mol2, hit_ats2, order2new, True, True, legend2)
        
        svmol1newiter, img1newiter = _show_common_core_gradient_write( mol1, hit_ats1, order1newiter, True, True, legendright)
        svmol2newiter, img2newiter = _show_common_core_gradient_write( mol2, hit_ats2, order2newiter, True, True, legendright)

        imo1, imo2 = convertgraphics(svmol1, svmol2)
        imo1new, imo2new = convertgraphics(svmol1new, svmol2new)
        imo1newiter, imo2newiter = convertgraphics(svmol1newiter, svmol2newiter)
        
        imagearray.append(imo1)
        imagearray.append(imo2)
        
             
        imagearraynew.append(imo1new)
        imagearraynew.append(imo2new)
        
             
        imagearraynewiter.append(imo1newiter)
        imagearraynewiter.append(imo2newiter)
        
        #SVG(svmol1new)
        #SVG(svmol2new)
        
        molnamescounter = molnamescounter + 2


list_of_dummy_atoms_idx
[]
nr_of_dummy_atoms
1
list_of_real_atoms_idx
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]
list_of_dummy_atoms_idx
[]
nr_of_dummy_atoms
1
list_of_real_atoms_idx
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]
list_of_dummy_atoms_idx
[6, 7, 8, 9, 10, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]
nr_of_dummy_atoms
24
list_of_real_atoms_idx
[0, 1, 2, 3, 4, 5, 11, 12, 13, 14, 15]
list_of_dummy_atoms_idx
[11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]
nr_of_dummy_atoms
13
list_of_real_atoms_idx
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Final mutation route:
[27, 28, 26, 33, 32, 29, 25, 31, 24, 30, 23, 21, 22, 20, 19]
Final mutation route:
[10, 9, 8, 7, 6]
Final mutation route:
[18, 17, 16]
Final mutation route:
[11]
Final mutation route:
[12]
Final mutation route:
[20, 21, 19, 22, 18, 17, 

list_of_dummy_atoms_idx
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 30, 31, 32, 33]
nr_of_dummy_atoms
29
list_of_real_atoms_idx
[24, 25, 26, 27, 28, 29]
list_of_dummy_atoms_idx
[0, 1, 2, 3, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]
nr_of_dummy_atoms
26
list_of_real_atoms_idx
[4, 5, 6, 7, 29, 30]
Final mutation route:
[10, 0, 9, 1, 8, 2, 3, 7, 18, 17, 4, 6, 16, 5, 12, 15, 11, 14, 13, 32, 33, 19, 21, 31, 20, 30, 22, 23]
Final mutation route:
[2, 0, 1, 3]
Final mutation route:
[23, 22, 21, 20, 18, 19, 17, 13, 14, 16, 25, 24, 12, 15, 28, 10, 11, 26, 27, 9, 8]
Final mutation route:
[0, 10, 1, 9, 2, 8, 3, 7, 17, 18, 16, 4, 6, 5, 12, 15, 11, 14, 13, 32, 33, 31, 19, 21, 20, 30, 22, 23]
Final mutation route:
[0, 2, 1, 3]
Final mutation route:
[22, 23, 21, 20, 19, 18, 17, 16, 14, 13, 24, 25, 15, 12, 10, 28, 11, 26, 9, 27, 8]
list_of_dummy_atoms_idx
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19

list_of_dummy_atoms_idx
[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]
nr_of_dummy_atoms
33
list_of_real_atoms_idx
[0, 1]
list_of_dummy_atoms_idx
[0, 1, 2, 3, 6, 7, 8, 9, 10, 11]
nr_of_dummy_atoms
11
list_of_real_atoms_idx
[4, 5]
Final mutation route:
[27, 28, 26, 33, 32, 29, 25, 31, 24, 17, 18, 30, 23, 10, 21, 16, 22, 9, 15, 20, 19, 14, 8, 7, 12, 13, 6, 11, 5, 4, 3, 2]
Final mutation route:
[3, 2, 0, 1]
Final mutation route:
[9, 10, 8, 11, 7, 6]
Final mutation route:
[27, 26, 28, 25, 29, 32, 33, 24, 31, 23, 30, 17, 18, 16, 22, 21, 10, 20, 9, 15, 8, 14, 19, 13, 12, 7, 11, 6, 5, 4, 3, 2]
Final mutation route:
[0, 2, 3, 1]
Final mutation route:
[9, 8, 10, 7, 11, 6]
list_of_dummy_atoms_idx
[11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]
nr_of_dummy_atoms
13
list_of_real_atoms_idx
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
list_of_dummy_atoms_idx
[6, 7, 8, 9, 10, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 3

list_of_dummy_atoms_idx
[0, 1, 2, 3, 4, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]
nr_of_dummy_atoms
21
list_of_real_atoms_idx
[5, 6, 7]
list_of_dummy_atoms_idx
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]
nr_of_dummy_atoms
38
list_of_real_atoms_idx
[20, 21, 22]
Final mutation route:
[20, 21, 19, 0, 18, 22, 1, 17, 14, 15, 16, 2, 3, 13, 4]
Final mutation route:
[12, 11, 10, 9, 8]
Final mutation route:
[0, 1, 37, 39, 36, 2, 38, 3, 35, 4, 6, 34, 5, 10, 32, 33, 7, 9, 30, 31, 12, 29, 8, 11, 14, 28, 13, 25, 27, 17, 24, 26, 15, 23, 16, 18, 19]
Final mutation route:
[20, 19, 21, 18, 22, 0, 17, 1, 2, 16, 14, 15, 13, 3, 4]
Final mutation route:
[11, 12, 10, 9, 8]
Final mutation route:
[0, 1, 37, 39, 2, 36, 35, 38, 3, 34, 4, 6, 5, 33, 32, 10, 31, 30, 7, 9, 8, 12, 29, 11, 14, 28, 13, 27, 25, 15, 17, 24, 26, 16, 23, 18, 19]
list_of_dummy_atoms_idx
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1

list_of_dummy_atoms_idx
[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]
nr_of_dummy_atoms
22
list_of_real_atoms_idx
[0, 1]
list_of_dummy_atoms_idx
[0, 1, 2, 3, 6, 7, 8, 9, 10, 11]
nr_of_dummy_atoms
11
list_of_real_atoms_idx
[4, 5]
Final mutation route:
[11, 12, 20, 10, 19, 21, 9, 18, 22, 7, 8, 17, 6, 14, 15, 16, 5, 13, 4, 3, 2]
Final mutation route:
[3, 2, 0, 1]
Final mutation route:
[9, 10, 8, 11, 7, 6]
Final mutation route:
[11, 12, 20, 10, 19, 21, 9, 18, 22, 17, 8, 7, 6, 16, 14, 15, 13, 5, 4, 3, 2]
Final mutation route:
[0, 2, 3, 1]
Final mutation route:
[9, 8, 10, 7, 11, 6]
list_of_dummy_atoms_idx
[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]
nr_of_dummy_atoms
22
list_of_real_atoms_idx
[0, 1]
list_of_dummy_atoms_idx
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]
nr_of_dummy_atoms
33
list_of_real_atoms_idx
[16, 18]
Final mutation route:
[12, 11, 14, 16,

...

list_of_dummy_atoms_idx
[]
nr_of_dummy_atoms
1
list_of_real_atoms_idx
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]
list_of_dummy_atoms_idx
[]
nr_of_dummy_atoms
1
list_of_real_atoms_idx
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]
list_of_dummy_atoms_idx
[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]
nr_of_dummy_atoms
20
list_of_real_atoms_idx
[0, 1, 2, 3]
list_of_dummy_atoms_idx
[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34]
nr_of_dummy_atoms
32
list_of_real_atoms_idx
[0, 1, 2, 3]
Final mutation route:
[18, 17, 16, 15, 8, 9, 10, 14, 7, 21, 13, 19, 11, 20, 6, 12, 5, 22, 4]
Final mutation route:
[28, 27, 25, 24, 26, 23, 19, 21, 22, 18, 20, 17, 15, 16, 13, 14, 12, 11, 30, 31, 10, 29, 9, 8, 7, 33, 34, 6, 32, 5, 4]
Final mutation route:
[16, 17, 18, 15, 8, 9, 10, 14, 19, 21, 7, 13, 11, 6, 12, 20, 22, 5, 4]
Final mutation 

list_of_dummy_atoms_idx
[19, 25, 27, 30, 31, 34]
nr_of_dummy_atoms
7
list_of_real_atoms_idx
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21, 22, 23, 24, 26, 28, 29, 32, 33]
list_of_dummy_atoms_idx
[0, 1, 3, 10, 11, 12, 13, 19, 20, 21, 22, 28, 29, 30, 31, 32, 33, 40, 41, 46, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86]
nr_of_dummy_atoms
59
list_of_real_atoms_idx
[2, 4, 5, 6, 7, 8, 9, 14, 15, 16, 17, 18, 23, 24, 25, 26, 27, 34, 35, 36, 37, 38, 39, 42, 43, 44, 45, 47, 48]
Final mutation route:
[34]
Final mutation route:
[19]
Final mutation route:
[25]
Final mutation route:
[27]
Final mutation route:
[30]
Final mutation route:
[31]
Final mutation route:
[0, 1]
Final mutation route:
[3]
Final mutation route:
[40]
Final mutation route:
[41]
Final mutation route:
[13, 12, 11, 10]
Final mutation route:
[46]
Final mutation route:
[76, 86, 85, 75, 83, 84, 77, 74, 8

list_of_dummy_atoms_idx
[0, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
nr_of_dummy_atoms
28
list_of_real_atoms_idx
[1, 2, 3, 4, 5, 32, 33, 34]
list_of_dummy_atoms_idx
[0, 1, 2, 3, 4, 5, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41]
nr_of_dummy_atoms
35
list_of_real_atoms_idx
[6, 7, 8, 9, 10, 29, 30, 31]
Final mutation route:
[0]
Final mutation route:
[6]
Final mutation route:
[28, 27, 25, 24, 26, 23, 19, 21, 22, 18, 20, 17, 15, 16, 13, 14, 12, 11, 30, 31, 10, 29, 9, 8, 7]
Final mutation route:
[23, 24, 22, 25, 21, 20, 18, 19, 16, 27, 15, 17, 13, 26, 14, 12, 28, 11]
Final mutation route:
[2, 0, 1, 3, 4, 5]
Final mutation route:
[32]
Final mutation route:
[39, 40, 38, 41, 37, 36, 35, 34, 33]
Final mutation route:
[0]
Final mutation route:
[6]
Final mutation route:
[25, 27, 28, 26, 24, 23, 22, 21, 19, 20, 18, 17, 16, 15, 14, 13, 12, 11, 10, 30, 31, 9, 29, 8, 7]
Final 

list_of_dummy_atoms_idx
[0, 1, 2, 3, 7, 8, 9, 10, 11]
nr_of_dummy_atoms
10
list_of_real_atoms_idx
[4, 5, 6]
list_of_dummy_atoms_idx
[0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42]
nr_of_dummy_atoms
41
list_of_real_atoms_idx
[4, 18, 20]
Final mutation route:
[3, 2, 0, 1]
Final mutation route:
[7, 8, 9, 10, 11]
Final mutation route:
[11, 10, 9, 8, 7]
Final mutation route:
[19]
Final mutation route:
[2, 0, 1, 3]
Final mutation route:
[15, 14, 16, 13, 17, 12, 11, 9, 10, 7, 8, 6, 5]
Final mutation route:
[34, 33, 32, 31, 30, 27, 29, 28, 26, 25, 23, 24, 22, 21, 40, 38, 35, 37, 39, 36, 41, 42]
Final mutation route:
[34, 33, 32, 31, 30, 40, 27, 29, 39, 26, 28, 41, 38, 25, 37, 42, 23, 24, 36, 22, 35, 21]
Final mutation route:
[0, 2, 3, 1]
Final mutation route:
[7, 8, 9, 10, 11]
Final mutation route:
[11, 10, 9, 8, 7]
Final mutation route:
[19]
Final mutation route:
[0, 2, 1, 3]
Final mutation

list_of_dummy_atoms_idx
[0, 1, 2, 3, 6, 7, 8, 9, 10, 11]
nr_of_dummy_atoms
11
list_of_real_atoms_idx
[4, 5]
list_of_dummy_atoms_idx
[0, 1, 2, 3, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41]
nr_of_dummy_atoms
41
list_of_real_atoms_idx
[4, 5]
Final mutation route:
[3, 2, 0, 1]
Final mutation route:
[9, 10, 8, 11, 7, 6]
Final mutation route:
[2, 0, 1, 3]
Final mutation route:
[23, 24, 22, 21, 25, 39, 20, 38, 40, 18, 19, 41, 37, 16, 15, 17, 27, 36, 35, 13, 26, 14, 34, 12, 28, 10, 32, 33, 11, 30, 31, 9, 8, 29, 7, 6]
Final mutation route:
[0, 2, 3, 1]
Final mutation route:
[9, 8, 10, 7, 11, 6]
Final mutation route:
[0, 2, 1, 3]
Final mutation route:
[23, 22, 24, 39, 38, 40, 21, 25, 20, 37, 41, 36, 19, 18, 16, 17, 27, 15, 35, 26, 14, 13, 28, 12, 34, 11, 10, 33, 32, 9, 31, 30, 29, 8, 7, 6]
list_of_dummy_atoms_idx
[0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11]
nr_of_dummy_atoms
12
list_of_real_atoms_idx
[4]
list

In [34]:
coresperfile = 40 * 2
filenumber = len(imagearray)/coresperfile
filenumber = math.ceil(filenumber)

In [35]:
w, h = imagearray[0].size

In [36]:
filename = ('MutationRoutes')
filenames = []
for x in range(0,filenumber):
    filenames.append(filename + str(x+1) + '.png')

In [39]:
count = 0 
for fname in filenames:
    new_image = PIL.Image.new('RGB', (int(w*3), int(h*(coresperfile))))
    

   # for imgi in range(len(imagearray)):
    for imgi in range(coresperfile):    
        new_image.paste(imagearray[count], (0,  imgi*h))
        new_image.paste(imagearraynew[count], (w,  imgi*h))
        new_image.paste(imagearraynewiter[count], (w*2,  imgi*h))
        count = count + 1
    
    new_image.save(fname)
